# Installs
### Uncomment to install dependencies

In [ ]:
# !python.exe -m pip install --upgrade pip
# !pip install pandas
# !pip install numpy
# !pip install altair
# !pip install "vegafusion[embed]"   
# !pip install vl-convert-python>=1.6.0
# !pip install pyarrow

# Imports

In [2]:
import pandas as pd
import numpy as np
import altair as alt
import vegafusion as vf

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

# EDA

In [3]:
# Load the dataset and show basic info
df = pd.read_csv("annotations.csv")
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49551 entries, 0 to 49550
Data columns (total 53 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      49551 non-null  int64 
 1   person_id                       49551 non-null  int64 
 2   filename                        49551 non-null  object
 3   class1                          49551 non-null  object
 4   class2                          3677 non-null   object
 5   bounding_box                    49551 non-null  object
 6   gender_presentation_masc        49551 non-null  int64 
 7   gender_presentation_fem         49551 non-null  int64 
 8   gender_presentation_non_binary  49551 non-null  int64 
 9   gender_presentation_na          49551 non-null  int64 
 10  skin_tone_1                     49551 non-null  int64 
 11  skin_tone_2                     49551 non-null  int64 
 12  skin_tone_3                     49551 non-null

,Unnamed: 0,person_id,gender_presentation_masc,gender_presentation_fem,gender_presentation_non_binary,gender_presentation_na,skin_tone_1,skin_tone_2,skin_tone_3,skin_tone_4,...,has_headscarf,has_eyeware,visible_torso,visible_face,visible_minimal,lighting_underexposed,lighting_dimly_lit,lighting_well_lit,lighting_na,lighting_overexposed
count,49551.000000,4.955100e+04,49551.000000,49551.000000,49551.000000,49551.000000,49551.000000,49551.000000,49551.000000,49551.000000,...,49551.000000,49551.000000,49551.000000,49551.000000,49551.000000,49551.000000,49551.000000,49551.000000,49551.000000,49551.000000
mean,24775.000000,1.307137e+15,0.670824,0.206757,0.001917,0.113055,0.130250,0.699824,0.940142,0.879659,...,0.046679,0.109624,0.728744,0.296099,0.148776,0.026498,0.224092,0.797158,0.017719,0.018991
std,14304.285931,1.293425e+15,0.469919,0.404984,0.043744,0.316664,0.443802,1.019083,1.101531,1.030116,...,0.210953,0.312424,0.444612,0.456540,0.355871,0.160612,0.416987,0.402120,0.131930,0.136493
min,0.000000,3.501556e+14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12387.500000,6.396669e+14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,24775.000000,8.475259e+14,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,37162.500000,1.314718e+15,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,1.000000,...,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,49550.000000,9.330696e+15,1.000000,1.000000,1.000000,1.000000,6.000000,6.000000,6.000000,6.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
# Check for missing values
df.isna().sum()

Unnamed: 0                            0
person_id                             0
filename                              0
class1                                0
class2                            45874
bounding_box                          0
gender_presentation_masc              0
gender_presentation_fem               0
gender_presentation_non_binary        0
gender_presentation_na                0
skin_tone_1                           0
skin_tone_2                           0
skin_tone_3                           0
skin_tone_4                           0
skin_tone_5                           0
skin_tone_6                           0
skin_tone_7                           0
skin_tone_8                           0
skin_tone_9                           0
skin_tone_10                          0
skin_tone_na                          0
age_presentation_young                0
age_presentation_older                0
age_presentation_middle               0
age_presentation_na                   0


In [5]:
df.columns

Index(['Unnamed: 0', 'person_id', 'filename', 'class1', 'class2',
       'bounding_box', 'gender_presentation_masc', 'gender_presentation_fem',
       'gender_presentation_non_binary', 'gender_presentation_na',
       'skin_tone_1', 'skin_tone_2', 'skin_tone_3', 'skin_tone_4',
       'skin_tone_5', 'skin_tone_6', 'skin_tone_7', 'skin_tone_8',
       'skin_tone_9', 'skin_tone_10', 'skin_tone_na', 'age_presentation_young',
       'age_presentation_older', 'age_presentation_middle',
       'age_presentation_na', 'hair_color_brown', 'hair_color_blonde',
       'hair_color_grey', 'hair_color_na', 'hair_color_black',
       'hair_color_colored', 'hair_color_red', 'hairtype_coily',
       'hairtype_dreadlocks', 'hairtype_bald', 'hairtype_straight',
       'hairtype_curly', 'hairtype_wavy', 'hairtype_na', 'has_facial_hair',
       'has_tattoo', 'has_cap', 'has_mask', 'has_headscarf', 'has_eyeware',
       'visible_torso', 'visible_face', 'visible_minimal',
       'lighting_underexposed', 'ligh

In [6]:
gender_cols = [
    "gender_presentation_masc", 
    "gender_presentation_fem", 
    "gender_presentation_non_binary", 
    "gender_presentation_na"
]

df[gender_cols].sum().reset_index(name="count").rename(columns={"index": "gender"})


,gender,count
0,gender_presentation_masc,33240
1,gender_presentation_fem,10245
2,gender_presentation_non_binary,95
3,gender_presentation_na,5602


In [7]:
alt.Chart(df[gender_cols].sum().reset_index(name="count").rename(columns={"index": "gender"})).mark_bar().encode(
    x="gender:N",
    y="count:Q",
    color="gender:N"
)


alt.Chart(...)

In [8]:
skin_cols = [c for c in df.columns if c.startswith("skin_tone_")]
skin_counts = df[skin_cols].sum().reset_index(name="count").rename(columns={"index": "skin_tone"})

alt.Chart(skin_counts).mark_bar().encode(
    x="skin_tone:N",
    y="count:Q",
    color="skin_tone:N"
)


alt.Chart(...)

In [9]:
age_cols = [c for c in df.columns if c.startswith("age_presentation_")]
age_counts = df[age_cols].sum().reset_index(name="count").rename(columns={"index": "age_group"})

alt.Chart(age_counts).mark_bar().encode(
    x="age_group:N",
    y="count:Q",
    color="age_group:N"
)


alt.Chart(...)

In [10]:
hair_color_cols = [c for c in df.columns if c.startswith("hair_color_")]
hair_counts = df[hair_color_cols].sum().reset_index(name="count").rename(columns={"index": "hair_color"})

alt.Chart(hair_counts).mark_bar().encode(
    x="hair_color:N",
    y="count:Q",
    color="hair_color:N"
)


alt.Chart(...)

In [11]:
hairtype_cols = [c for c in df.columns if c.startswith("hairtype_")]
hairtype_counts = df[hairtype_cols].sum().reset_index(name="count").rename(columns={"index": "hair_type"})

alt.Chart(hairtype_counts).mark_bar().encode(
    x="hair_type:N",
    y="count:Q",
    color="hair_type:N"
)


alt.Chart(...)

In [12]:
vis_cols = ["visible_torso", "visible_face", "visible_minimal"]
vis_counts = df[vis_cols].sum().reset_index(name="count").rename(columns={"index": "visibility"})

alt.Chart(vis_counts).mark_bar().encode(
    x="visibility:N",
    y="count:Q",
    color="visibility:N"
)


alt.Chart(...)

In [13]:
lighting_cols = [c for c in df.columns if c.startswith("lighting_")]
lighting_counts = df[lighting_cols].sum().reset_index(name="count").rename(columns={"index": "lighting"})

alt.Chart(lighting_counts).mark_bar().encode(
    x="lighting:N",
    y="count:Q",
    color="lighting:N"
)


alt.Chart(...)

In [14]:
# Gender masc + skin tone distribution
subset = df[df["gender_presentation_masc"] == 1]
skin_counts_masc = subset[skin_cols].sum().reset_index(name="count").rename(columns={"index": "skin_tone"})

alt.Chart(skin_counts_masc).mark_bar().encode(
    x="skin_tone:N",
    y="count:Q",
    color="skin_tone:N"
).properties(title="Skin tone distribution (masculine-presenting)")


alt.Chart(...)

In [15]:
alt.Chart(df).mark_bar().encode(
    x="class1:N",
    y="count()",
    color="class1:N"
).properties(title="Distribution of class1")


alt.Chart(...)

In [16]:
gender_cols = [
    "gender_presentation_masc", 
    "gender_presentation_fem", 
    "gender_presentation_non_binary", 
    "gender_presentation_na"
]

# Melt to long format for Altair
gender_long = df.melt(id_vars=["class1"], value_vars=gender_cols, var_name="gender", value_name="present")
gender_long = gender_long[gender_long["present"] == 1]

alt.Chart(gender_long).mark_bar().encode(
    x="gender:N",
    y="count()",
    color="class1:N"
).properties(title="class1 vs Gender Presentation")


alt.Chart(...)

In [17]:
skin_cols = [c for c in df.columns if c.startswith("skin_tone_")]
skin_long = df.melt(id_vars=["class1"], value_vars=skin_cols, var_name="skin_tone", value_name="present")
skin_long = skin_long[skin_long["present"] == 1]

alt.Chart(skin_long).mark_bar().encode(
    x="skin_tone:N",
    y="count()",
    color="class1:N"
).properties(title="class1 vs Skin Tone")


alt.Chart(...)

In [18]:
age_cols = [c for c in df.columns if c.startswith("age_presentation_")]
age_long = df.melt(id_vars=["class1"], value_vars=age_cols, var_name="age_group", value_name="present")
age_long = age_long[age_long["present"] == 1]

alt.Chart(age_long).mark_bar().encode(
    x="age_group:N",
    y="count()",
    color="class1:N"
).properties(title="class1 vs Age Presentation")


alt.Chart(...)

In [19]:
hair_color_cols = [c for c in df.columns if c.startswith("hair_color_")]
hair_color_long = df.melt(id_vars=["class1"], value_vars=hair_color_cols, var_name="hair_color", value_name="present")
hair_color_long = hair_color_long[hair_color_long["present"] == 1]

alt.Chart(hair_color_long).mark_bar().encode(
    x="hair_color:N",
    y="count()",
    color="class1:N"
).properties(title="class1 vs Hair Color")


alt.Chart(...)

In [20]:
heatmap_data = (
    skin_long.groupby(["class1", "skin_tone"]).size().reset_index(name="count")
)

alt.Chart(heatmap_data).mark_rect().encode(
    x="skin_tone:N",
    y="class1:N",
    color="count:Q"
).properties(title="Heatmap of class1 × Skin Tone")


alt.Chart(...)